In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import tqdm
from scipy.stats import spearmanr, kendalltau

tqdm.pandas()

## Task 1:  Data preprocessing

In [2]:
players = pd.read_pickle('players.pkl')

In [3]:
pd.DataFrame.from_records(list(players.values())).to_parquet('players.parquet')
del players

#### Обрабатываем метаданные турниров

In [4]:
tournaments = pd.read_pickle('tournaments.pkl')

In [5]:
def len_or_zero(x):
    return 0 if x is None else len(x)

def sum_values_or_zero(x):
    return 0 if x is None else sum(x.values())

def parse_tournament(tournament):
    return {'id': tournament.get('id'),
        'name': tournament.get('name'),
        'dateStart': tournament.get('dateStart'),
        'dateEnd': tournament.get('dateEnd'),
        'type_name': tournament.get('type').get('name'),
        'season': tournament.get('season'),
        'num_rounds': len_or_zero(tournament.get('questionQty', list())),
        'num_questions': sum_values_or_zero(tournament.get('questionQty'))
    }

In [7]:
tournaments_records = []
for tournament in tqdm(tournaments.values()):
    year = tournament['dateStart'].split('-')[0]
    if year == '2019' or year == '2020':
        tournaments_records.append(parse_tournament(tournament))

del tournaments

  0%|          | 0/5528 [00:00<?, ?it/s]

In [8]:
pd.DataFrame.from_records(tournaments_records).to_parquet('tournaments.parquet')

#### Обрабатываем результаты турниров

In [9]:
results = pd.read_pickle('results.pkl')

In [10]:
def get_value_at(x, path):
    for item in path:
        if item in x:
            x = x[item]
            if x is None:
                return x
        else:
            return None
    return x
        
def parse_team_member(team_member):
    return {
        'player_id': get_value_at(team_member, ['player', 'id']),
        'rating': team_member.get('rating'),
        'usedRating': team_member.get('usedRating')
    }

def parse_team_result(team_result):
    team_info = {
        'team_id': get_value_at(team_result, ['team', 'id']),
        'team_name': get_value_at(team_result, ['team', 'name']),
        'team_town_id': get_value_at(team_result, ['team', 'town', 'id']),
        'team_town_name': get_value_at(team_result, ['team', 'town', 'name']),
        'mask': team_result.get('mask'),
        'questionsTotal': team_result.get('questionsTotal'),
        'position': team_result.get('position'),
    }
    team_info
    result = []
    for team_member in team_result.get('teamMembers'):
        result.append({
            **team_info,
            **parse_team_member(team_member)
        })
    return result

def parse_result(result):
    records = []
    for team_result in result:
        records.extend(parse_team_result(team_result))
    return records

In [11]:
df_results = []

for tournament_id, result in tqdm(results.items()):
    records = parse_result(result)
    df_result = pd.DataFrame.from_records(records)
    df_result['tournament_id'] = tournament_id
    df_results.append(df_result)
    
del results

  0%|          | 0/5528 [00:00<?, ?it/s]

In [12]:
df_results = pd.concat(df_results, axis=0)

In [14]:
df_results['team_id'] = df_results['team_id'].astype(np.int64)
df_results['team_town_id'] = df_results['team_town_id'].fillna(-1).astype(np.int64)
df_results['questionsTotal'] = df_results['questionsTotal'].fillna(0).astype(np.int64)
df_results['player_id'] = df_results['player_id'].astype(np.int64)
df_results['rating'] = df_results['rating'].astype(np.int64)
df_results['usedRating'] = df_results['usedRating'].astype(np.int64)

In [15]:
df_results.to_parquet('results.parquet')

## Task 2-3: baseline model

In [2]:
df_results = pd.read_parquet('./results.parquet')
df_tournaments = pd.read_parquet('./tournaments.parquet')
df_players = pd.read_parquet('./players.parquet')

# Используем только турниры, по которым есть метаинформация
df_results = df_results[df_results['tournament_id'].isin(df_tournaments['id'])]

# Используем турниры с непустой маской
df_results = df_results[~df_results['mask'].isna()].reset_index(drop=True)

# Используем турниры типов: 'Обычный', 'Синхрон', 'Строго синхронный'
tournaments_id_use = df_tournaments.loc[df_tournaments['type_name'].isin(['Обычный', 'Синхрон', 'Строго синхронный']),'id'].values
df_results = df_results[df_results['tournament_id'].isin(tournaments_id_use)].reset_index(drop=True)

# Объединяем результаты с метаинформацией по турнирам
df_results = df_results.merge(df_tournaments, left_on='tournament_id', right_on='id', how='left')
df_results = df_results.sort_values('dateStart').reset_index(drop=True)
df_results['year_start'] = df_results['dateStart'].apply(lambda x: int(x.split("-")[0]))

# Используем данные по турнирам за 2019 и 2020 годы
df_results = df_results[df_results['year_start'].isin([2019, 2020])].reset_index(drop=True)

Сделаем упрощающее предположение, что сложность вопроса оценивается долей команд в турнире, которые на него верно ответили.

In [3]:
def nth_game(x, n):
    if n < len(x):
        return int(x[n].replace('X','0').replace('?','0'))
    else:
        return 0
    
def questions_difficulty(data):
    year_start = data['year_start'].values[0]
    df = data[['team_id', 'mask']].drop_duplicates()
    num_questions = df['mask'].apply(len).max()

    proportions = []
    for i in range(num_questions):
        proportions.append(df['mask'].apply(lambda x: nth_game(x, i)).mean())
    df_res = pd.DataFrame()
    df_res['question_order'] = np.arange(num_questions)
    df_res['difficulty'] = proportions
    df_res['year_start'] = year_start
    return df_res

In [4]:
df_questions = df_results.groupby('tournament_id').progress_apply(questions_difficulty).reset_index().drop(columns=['level_1'])

  0%|          | 0/763 [00:00<?, ?it/s]

In [5]:
df_questions.head()

,tournament_id,question_order,difficulty,year_start
0,4772,0,0.883117,2019
1,4772,1,0.779221,2019
2,4772,2,0.445887,2019
3,4772,3,0.519481,2019
4,4772,4,0.878788,2019


Предположим, что 
* сила игрока выражается его рейтингом к моменту начала турнира (rating) (на самом деле на рейтинг влияют не только знания конкретного игрока, но и общая сила команд, в которых он играл),
* если команда ответила на вопрос, то каждый из игроков ответил на вопрос (на самом деле мы не знаем, кто именно из команды ответил)

In [6]:
def player_result(df):
    mask = df['mask'].values[0]
    num_questions = len(mask)
    mask_list = [nth_game(mask, i) for i in range(num_questions)]
    df_res = pd.DataFrame()
    df_res['question_order'] = np.arange(num_questions)
    df_res['target'] = mask_list
    df_res['player_id'] = df['player_id'].values[0]
    df_res['tournament_id'] = df['tournament_id'].values[0]
    df_res['rating'] = df['rating'].values[0]
    df_res['position'] = df['position'].values[0]
    return df_res

In [7]:
df_results_sample = df_results.sample(400000)

df_player_results = df_results_sample.groupby(df_results_sample.index).progress_apply(player_result).reset_index(drop=True)

  0%|          | 0/400000 [00:00<?, ?it/s]

In [8]:
df_player_results.head()

,question_order,target,player_id,tournament_id,rating,position
0,0,1,168063,5266,1340,10.0
1,1,1,168063,5266,1340,10.0
2,2,1,168063,5266,1340,10.0
3,3,0,168063,5266,1340,10.0
4,4,0,168063,5266,1340,10.0


Объединим информацию об ответах на вопросы с информацией по игрокам.

In [9]:
df_res = df_questions.merge(df_player_results, how='inner', on=['tournament_id', 'question_order'], validate='1:m')

In [10]:
df_res.head()

,tournament_id,question_order,difficulty,year_start,target,player_id,rating,position
0,4772,0,0.883117,2019,1,6212,13507,1.0
1,4772,0,0.883117,2019,1,173294,375,228.5
2,4772,0,0.883117,2019,1,173292,375,228.5
3,4772,0,0.883117,2019,1,145477,459,228.5
4,4772,0,0.883117,2019,1,145834,536,228.5


In [11]:
train = df_res[df_res['year_start'] == 2019].dropna().reset_index(drop=True)
test = df_res[df_res['year_start'] == 2020].dropna().reset_index(drop=True)

В качестве признаков выступают рейтинг игрока и сложность вопроса, а в качестве таргета ответил ли игрок на вопрос.
В качетсве baseline модели используем логистическую регрессию.

In [12]:
features = ['rating', 'difficulty']

X_train = train[features]
Y_train = train['target']
X_test = test[features]
Y_test = test['target']

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
linear_model = LogisticRegression()
linear_model.fit(X_train_scaled, Y_train)

test['target_pred'] = linear_model.predict_proba(X_test_scaled)[:, 1]

In [14]:
# Как игрок в среднем отвечает во время турнира
test_pred = test.groupby(['player_id', 'tournament_id']).agg(target_pred_mean=('target_pred', 'mean')).reset_index()
test_true = test.groupby(['player_id', 'tournament_id']).first().reset_index()
test_res = test_true.merge(test_pred, how='left')

### Метрики baseline модели

In [15]:
corr_spearman = test_res.groupby('tournament_id').apply(lambda x: spearmanr(x['target_pred_mean'], x['position'])[0])
print(f'Spearman correlation = {-corr_spearman.mean():.4f}')

corr_kendall = test_res.groupby('tournament_id').apply(lambda x: kendalltau(x['target_pred_mean'], x['position'])[0])
print(f'Kendall correlation = {-corr_kendall.mean():.4f}')

Spearman correlation = 0.7133
Kendall correlation = 0.5522


## Task 4: EM-algorithm

In [16]:
def nth_game(x, n):
    if n < len(x):
        return int(x[n].replace('X','0').replace('?','0'))
    else:
        return 0

In [17]:
df_tournaments['offset'] = df_tournaments['num_questions'].cumsum()

df_offset = df_results[['tournament_id', 'num_questions']].groupby('tournament_id').first().reset_index()
df_offset['offset'] = df_offset['num_questions'].cumsum()
df_offset = df_offset[['tournament_id', 'offset']]

df_results = df_results.merge(df_offset)

In [18]:
# Проиндексируем вопросы по всем турнирам и построим датасет, как каждый игрок отвечает на каждый вопрос,
# в качестве индекса игрока используется player_id
def player_questions(data):
    player_id = data['player_id'].values[0]
    max_question_id = data['offset'].values[0]
    num_questions = data['num_questions'].values[0]
    min_question_id = max_question_id - data['num_questions'].values[0]
    mask = data['mask'].values[0]
    year_start = data['year_start'].values[0]
    tournament_id = data['tournament_id'].values[0]
    team_id = data['team_id'].values[0]
    records = []
    for i in range(num_questions):
        records.append((player_id, i + min_question_id, nth_game(mask, i), year_start, tournament_id, team_id))
        
    return pd.DataFrame(records, columns=['player_id', 'question_id', 'target', 'year_start', 'tournament_id', 'team_id'])

In [19]:
df_player_question = df_results.groupby(df_results.index).progress_apply(player_questions).reset_index().drop(columns=['level_0', 'level_1'])

  0%|          | 0/447488 [00:00<?, ?it/s]

In [20]:
df_player_question.head()

,player_id,question_id,target,year_start,tournament_id,team_id
0,144692,2189,1,2019,5266,66342
1,144692,2190,0,2019,5266,66342
2,144692,2191,1,2019,5266,66342
3,144692,2192,1,2019,5266,66342
4,144692,2193,0,2019,5266,66342


In [21]:
df_player_question.to_parquet('player_question.parquet')

In [22]:
df_player_question = pd.read_parquet('player_question.parquet')

In [36]:
train = df_player_question[df_player_question['year_start'] == 2019].reset_index(drop=True)

In [37]:
train.head()

,player_id,question_id,target,year_start,tournament_id,team_id
0,144692,2189,1,2019,5266,66342
1,144692,2190,0,2019,5266,66342
2,144692,2191,1,2019,5266,66342
3,144692,2192,1,2019,5266,66342
4,144692,2193,0,2019,5266,66342


In [38]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression

def init_z(target):
    zs = np.random.rand(*target.shape)
    zs[target == 0] = 0
    return zs

# Фиксируем силу игрока, сложность вопроса и вычисляем ожидания скрытых переменных E(z)
def e_step(data):
    data = data.copy()
    data['1_pred'] = 1 - data['pred']
    denom = (1 - data.groupby(['team_id', 'question_id'])['1_pred'].prod()).reset_index()
    denom.columns = ['team_id', 'question_id', 'denom']
    data = data.merge(denom, how='left')
    result = data['pred'] / data['denom']
    result[data['target'] == 0] = 0
    return result

# Обучаем логистическую модель при фиксированных E(z)
def m_step(data):
    data = data.copy()
    data_extra = data.copy()
    data['class'] = 1
    data['weight'] = data['zs']
    data_extra['class'] = 0
    data_extra['weight'] = 1 - data['zs']
    data_all = pd.concat([data, data_extra], axis=0)
    model = LogisticRegression()
    model.fit(train_all_features, data_all['class'], sample_weight=data_all['weight'])
    return model.predict_proba(train_features)[:, 1], model

In [39]:
train['zs'] = init_z(train['target'])

train_extra = train.copy()
train['class'] = 1
train['weight'] = train['zs']
train_extra['class'] = 0
train_extra['weight'] = 1 - train['zs']
train_all = pd.concat([train, train_extra], axis=0)
train_all.shape

(29324218, 9)

Рассчитаем все фичи по игрокам и вопросам через OneHotEncoder

In [40]:
ohe_player = OneHotEncoder()
first = ohe_player.fit_transform(train_all[['player_id']])
ohe_question = OneHotEncoder()
second = ohe_question.fit_transform(train_all[['question_id']])
train_all_features = hstack([first, second])

first = ohe_player.transform(train[['player_id']])
second = ohe_question.transform(train[['question_id']])
train_features = hstack([first, second])

In [41]:
train['pred'], model = m_step(train[['zs']])

C:\Users\s-ir\.julia\conda\3\envs\ml2\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
for i in tqdm(range(2)):
    train['zs'] = e_step(train)
    train['pred'], model = m_step(train[['zs']])

  0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\s-ir\.julia\conda\3\envs\ml2\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\s-ir\.julia\conda\3\envs\ml2\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

In [43]:
df_ranking = pd.DataFrame()
df_ranking['player_id'] = ohe_player.categories_[0]
df_ranking['rank'] = model.coef_[0][:len(ohe_player.categories_[0])]

In [44]:
df_ranking.sort_values(by='rank').tail(20)

,player_id,rank
6592,56647,2.231587
1894,13689,2.243943
978,7008,2.258806
9134,87637,2.263253
3871,27822,2.273858
3183,22799,2.287421
3623,26089,2.287840
2365,16837,2.298317
1488,10660,2.306190
4190,30270,2.310150


In [45]:
em_test_res = test.merge(df_ranking, on='player_id', how='left')

In [46]:
corr_spearman = em_test_res.groupby('tournament_id').apply(lambda x: spearmanr(x['rank'], x['position'])[0])
print(f'Spearman correlation = {-corr_spearman.mean():.4f}')

corr_kendall = em_test_res.groupby('tournament_id').apply(lambda x: kendalltau(x['rank'], x['position'])[0])
print(f'Kendall correlation = {-corr_kendall.mean():.4f}')

Spearman correlation = 0.6787
Kendall correlation = 0.5480


### Топовые игроки

In [53]:
df_ranking.merge(df_players, left_on='player_id', right_on='id', how='left').sort_values(by='rank', ascending=False).head(20)

,player_id,rank,id,name,patronymic,surname
3800,27403,2.720250,27403,Максим,Михайлович,Руссо
4185,30260,2.569439,30260,Евгений,Павлович,Спектор
598,4270,2.481592,4270,Александра,Владимировна,Брутер
3986,28751,2.479153,28751,Иван,Николаевич,Семушин
4170,30152,2.443810,30152,Артём,Сергеевич,Сорожкин
8003,74001,2.405634,74001,Игорь,Викторович,Мокин
2827,20207,2.378897,20207,Михаил,Леонидович,Матвеев
3842,27622,2.360168,27622,Николай,Георгиевич,Рябых
3000,21487,2.329042,21487,Борис,Яковлевич,Моносов
3033,21698,2.310540,21698,Александр,Владимирович,Мосягин


## Task 5: Question difficulty without ML

In [54]:
def nth_game(x, n):
    if n < len(x):
        return int(x[n].replace('X','0').replace('?','0'))
    else:
        return 0

def questions_difficulty(tournament_id):
    df = df_results.loc[df_results['tournament_id'] == tournament_id, ['team_id', 'mask']].drop_duplicates()
    very_easy = 0
    easy = 0
    medium = 0
    hard = 0
    very_hard = 0
    num_questions = df_tournaments.loc[df_tournaments['id'] == tournament_id, 'num_questions'].values[0]

    for i in range(num_questions):
        proportion = df['mask'].apply(lambda x: nth_game(x, i)).mean()
        if proportion >= 0.8:
            very_easy += 1
        elif proportion >= 0.6:
            easy += 1
        elif proportion >= 0.4:
            medium += 1
        elif proportion >= 0.2:
            hard += 1
        else:
            very_hard += 1

    return {
        'tournament_id': tournament_id,
        'very_easy': very_easy / num_questions,
        'easy': easy / num_questions,
        'medium': medium / num_questions,
        'hard': hard / num_questions,
        'very_hard': very_hard / num_questions,
    }

tournament_difficulty = [questions_difficulty(tournament_id) for tournament_id in df_results['tournament_id'].unique()]
df_tournament_difficulty = pd.DataFrame(tournament_difficulty)
df_results = df_results.merge(df_tournament_difficulty, how='left', on='tournament_id')

In [55]:
df_results[['id', 'name', 'very_hard']].drop_duplicates().sort_values('very_hard', ascending=False).head(20)

,id,name,very_hard
258851,6149,Чемпионат Санкт-Петербурга. Первая лига,1.000000
169927,5717,Чемпионат Таджикистана,0.844444
385432,6204,Второй Карагандинский марафон,0.817708
24560,5564,Молодёжный чемпионат Нижегородской области,0.694444
354217,6161,Студенческий чемпионат Тюменской области,0.666667
277408,5976,Открытый Студенческий чемпионат Краснодарского...,0.666667
345660,6027,ТрЭК-13,0.644444
115073,5599,Чемпионат Туркменистана,0.633333
22931,5426,Зимник,0.625000
105093,5453,Кубок губернатора Иркутской области,0.622222
